In [1]:
org_path="/mnt/Velocity_Vault/Code/Depth_Estimation/"

test_input_path=org_path+"Marigold/test_folders/test_images/"
test_output_path=org_path+"Marigold/test_folders/test_outputs/"
test_3d_path=org_path+"Marigold/test_folders/test_3d/"

In [2]:
import diffusers
from PIL import Image
import torch
import numpy as np
from tqdm import tqdm
from pprint import pprint
import cv2

depth_pipe = diffusers.MarigoldDepthPipeline.from_pretrained("prs-eth/marigold-depth-lcm-v1-0",
                                                        variant="fp16",
                                                        torch_dtype=torch.float16).to("cuda")

# normal_pipe = diffusers.MarigoldNormalsPipeline.from_pretrained("prs-eth/marigold-depth-lcm-v1-0",
#                                                         variant="fp16",
#                                                         torch_dtype=torch.float16).to("cuda")


/mnt/Extra/Virtual_Environment/Marigold/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/mnt/Extra/Virtual_Environment/Marigold/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [3]:
def downscale_image(image, target):
    """
    Downscales an image by a given scale factor.

    Args:
        image (numpy.ndarray): Input image array (H, W) or (H, W, C).
        scale (float): Scale factor (e.g., 0.5 for half size, 2 for double).

    Returns:
        numpy.ndarray: Resized image.
    """
    
    
    x_res=image.shape[1]
    y_res=image.shape[0]
    
    if target>=max(x_res,y_res):
        return image
    
    factor=0
    
    if x_res==max(x_res,y_res):
        factor=x_res/target
        x_res=target
        y_res=y_res/factor
        
    elif y_res==max(x_res,y_res):
        factor=y_res/target
        y_res=target
        x_res=x_res/factor
    

    new_size = (int(x_res), int(y_res))  # (width, height)
    
    downscaled = cv2.resize(image, new_size, interpolation=cv2.INTER_LINEAR)
    
    return downscaled

In [4]:
# uncertainity=depth.uncertainty[0]
# from pprint import pprint
# # pprint(uncertainity)

# import numpy as np
# import matplotlib.pyplot as plt

# def plot_array(data, cmap="viridis"):
#     """
#     Plots a 2D array with values ranging from 0 to 1.
    
#     Parameters:
#         data (numpy.ndarray): 2D array with values in [0,1].
#         cmap (str): Colormap for visualization.
#     """
#     plt.imshow(data, cmap=cmap, vmin=0, vmax=np.mean(data))
#     plt.colorbar(label="Value")
#     plt.axis("off")  # Hide axes
#     plt.show()

# plot_array(uncertainity)

In [5]:
# import numpy as np
# import trimesh
# import open3d as o3d
# import matplotlib.pyplot as plt
# from PIL import Image

# def create_3d_mesh(depth_map, color_image=None, scale=1.0, threshold=0.1,apply_colour=True):
#     """
#     Generates a 3D mesh from a depth map and applies color from the original image.

#     Parameters:
#         depth_map (numpy.ndarray): 2D array of depth values in the range [0,1].
#         color_image (numpy.ndarray): Corresponding 2D image (H, W, 3) with RGB colors.
#         scale (float): Factor to scale depth values.
#         threshold (float): Maximum depth difference allowed between nearby points for face creation.

#     Returns:
#         open3d.geometry.TriangleMesh: The 3D mesh object.
#     """
    
#     H, W = depth_map.shape
#     vertices = []
#     colors = []
#     faces = []

#     # Generate vertices and colors
#     for y in tqdm(range(H)):
#         for x in range(W):
#             z = depth_map[y, x] * scale  # Scale the depth
#             vertices.append([x, -y, z])  # Flip Y-axis for correct orientation
#             colors.append(color_image[y, x] / 255.0)  # Normalize RGB

#     vertices = np.array(vertices)
#     colors = np.array(colors)

#     # Generate faces (triangles)
#     def get_index(x, y):
#         return y * W + x  # Compute index in a 1D list

#     for y in tqdm(range(H - 1)):
#         for x in range(W - 1):
#             i1 = get_index(x, y)
#             i2 = get_index(x + 1, y)
#             i3 = get_index(x, y + 1)
#             i4 = get_index(x + 1, y + 1)

#             # Check depth differences
#             if abs(depth_map[y, x] - depth_map[y, x + 1]) < threshold and \
#                 abs(depth_map[y, x] - depth_map[y + 1, x]) < threshold:
#                 faces.append([i1, i2, i3])  # Triangle 1

#             if abs(depth_map[y + 1, x + 1] - depth_map[y, x + 1]) < threshold and \
#                 abs(depth_map[y + 1, x + 1] - depth_map[y + 1, x]) < threshold:
#                 faces.append([i2, i4, i3])  # Triangle 2

#     faces = np.array(faces)

#     # Create Open3D mesh
#     mesh = o3d.geometry.TriangleMesh()
#     mesh.vertices = o3d.utility.Vector3dVector(vertices)
#     mesh.triangles = o3d.utility.Vector3iVector(faces)
#     if apply_colour:
#         mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

#     return mesh

# # Load depth map and color image
# # depth_map = np.array(Image.open("depth_map.png").convert("L")) / 255.0  # Normalize [0,1]
# # color_image = np.array(Image.open("color_image.png"))

# image_array=np.array(image)

# # Generate 3D mesh
# mesh = create_3d_mesh(depth_array, color_image=image_array, scale=700, threshold=10e-3,apply_colour=False)

# # Save and visualize
# image_3d_name=image_name[:image_name.rfind('.')]+".ply"
# o3d.io.write_triangle_mesh(test_3d_path+image_3d_name, mesh)
# print(f"Saved as {image_3d_name}")
# # o3d.visualization.draw_geometries([mesh])


In [6]:
import numpy as np
import open3d as o3d
from tqdm import tqdm
from scipy.spatial import KDTree 

def fill_3d_mesh(depth_map, origin=(0,0,0),color_image=None, scale=1.0, threshold=0.1, euclid_threshold=1.5, apply_colour=True):
    """
    Generates a 3D mesh from a depth map and applies color from the original image.
    Filters out unused points and connects points within a Euclidean distance using KD-Tree.

    Parameters:
        depth_map (numpy.ndarray): 2D array of depth values in the range [0,1].
        color_image (numpy.ndarray): Corresponding 2D image (H, W, 3) with RGB colors.
        scale (float): Factor to scale depth values.
        threshold (float): Maximum depth difference allowed between nearby points for face creation.
        euclid_threshold (float): Maximum Euclidean distance to create additional triangles.
        apply_colour (bool): Whether to apply vertex colors.

    Returns:
        open3d.geometry.TriangleMesh: The 3D mesh object.
    """
    
    H, W = depth_map.shape
    vertices = []
    colors = []
    faces = []
    index_map = np.full((H, W), -1, dtype=int)  # Keeps track of valid vertices
    
    vertices.append([origin[0], -origin[1], origin[2]])

    # Generate valid vertices and store their index mapping
    for y in tqdm(range(H)):
        for x in range(W):
            z = depth_map[y, x] * scale  # Scale depth
            if z > 0:  # Ensure valid depth
                index_map[y, x] = len(vertices)  # Store new index
                vertices.append([x+origin[0], -(y+origin[1]), z+origin[2]])  # Flip Y-axis for correct orientation
                if apply_colour:
                    colors.append(color_image[y, x] / 255.0)  # Normalize RGB

    vertices = np.array(vertices)
    colors = np.array(colors) if apply_colour else None

    # Generate faces with threshold check
    for y in tqdm(range(H - 1)):
        for x in range(W - 1):
            i1 = index_map[y, x]
            i2 = index_map[y, x + 1]
            i3 = index_map[y + 1, x]
            i4 = index_map[y + 1, x + 1]

            if i1 == -1 or i2 == -1 or i3 == -1 or i4 == -1:
                continue  # Skip invalid points

            # Check depth differences
            if abs(depth_map[y, x] - depth_map[y, x + 1]) < threshold and \
               abs(depth_map[y, x] - depth_map[y + 1, x]) < threshold:
                faces.append([i1, i2, i3])  # Triangle 1

            if abs(depth_map[y + 1, x + 1] - depth_map[y, x + 1]) < threshold and \
               abs(depth_map[y + 1, x + 1] - depth_map[y + 1, x]) < threshold:
                faces.append([i2, i4, i3])  # Triangle 2

    # **Optimized Neighbor Search Using KD-Tree**
    # print("Building KD-Tree for fast neighbor search...")
    tree = KDTree(vertices)  # Precompute KD-Tree

    # print("Finding additional connections...")
    for i in tqdm(range(len(vertices))):
        # Find neighbors within euclid_threshold
        indices = tree.query_ball_point(vertices[i], euclid_threshold)
        for j in indices:
            if j > i:  # Avoid duplicate triangles
                faces.append([i, j, (i + 1) % len(vertices)])

    faces = np.array(faces)

    # Create Open3D mesh
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    if apply_colour:
        mesh.vertex_colors = o3d.utility.Vector3dVector(colors)

    return mesh




# import open3d as o3d
# import numpy as np

# def transpose_ply_axes(ply_path, output_path, axis_order):
#     # Load the PLY file
#     mesh = o3d.io.read_triangle_mesh(ply_path)
#     vertices = np.asarray(mesh.vertices)
    
#     # Apply the axis transformation
#     vertices = vertices[:, list(axis_order)]  # Reorder axes

#     # Update the mesh with new vertex positions
#     mesh.vertices = o3d.utility.Vector3dVector(vertices)

#     # Save the modified PLY file
#     o3d.io.write_triangle_mesh(output_path, mesh)

# # Example usage:
# # transpose_ply_axes(test_3d_path+image_3d_name, test_3d_path+image_3d_name, (0, 2, 1))  # Swap X ↔ Z




# Example usage
# depth_map = np.array(...)  # Load depth map (H, W)
# color_image = np.array(...)  # Load color image (H, W, 3)
# mesh = fill_3d_mesh(depth_map, color_image, scale=10, threshold=0.1, euclid_threshold=1.5)
# o3d.visualization.draw_geometries([mesh])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
# Segmentation (Very High RAM usage)
# import trimesh
# import numpy as np
# import random
# from tqdm import tqdm

# def segment_and_color_mesh(mesh_path, output_path="segmented_mesh.obj"):
#     """
#     Segments a 3D mesh into separate components and colors each part differently.

#     Parameters:
#         mesh_path (str): Path to the input 3D model file.
#         output_path (str): Path to save the segmented model.
#         threshold (float): Distance threshold for considering components as separate.
    
#     Returns:
#         trimesh.Trimesh: The segmented and colored mesh.
#     """

#     # Load mesh
    
#     mesh = trimesh.load_mesh(mesh_path)

#     # Find connected components
#     mesh.merge_vertices()
#     components = mesh.split(only_watertight=False)  # Allow non-watertight parts

#     print(f"Found {len(components)} separate objects in the mesh.")

#     # Assign colors to each component
#     colored_meshes = []
#     for comp in components:
#         if len(comp.vertices) == 0:
#             continue  # Skip empty components

#         # Generate a random color for each part
#         color = np.array([random.random(), random.random(), random.random(), 1.0])  # RGBA
#         comp.visual.vertex_colors = color  # Apply color

#         colored_meshes.append(comp)

#     # Merge all parts back into one mesh
#     segmented_mesh = trimesh.util.concatenate(colored_meshes)

#     # Save the colored segmented mesh
#     segmented_mesh.export(output_path)
#     print(f"Segmented mesh saved to {output_path}")

#     return segmented_mesh

# # Example usage:
# # segmented_mesh = segment_and_color_mesh("your_model.ply", "output.ply", threshold=50)


# # Example usage
# # mesh = highlight_disconnected_parts("your_model.ply", threshold=200)
# # o3d.visualization.draw_geometries([mesh])


# seperate_3d_name="disconnected_"+image_name[:image_name.rfind('.')]+".ply"

# mesh = segment_and_color_mesh(test_3d_path+image_3d_name, output_path=test_3d_path+seperate_3d_name)


In [8]:
# import open3d as o3d
# import numpy as np

# def read_vertices_from_ply(ply_file):
#     mesh = o3d.io.read_triangle_mesh(ply_file)
#     vertices = np.asarray(mesh.vertices)  # Convert to NumPy array
#     return vertices

# # Example usage
# ply_file = test_3d_path+"filled_leg_up.ply"
# vertices = read_vertices_from_ply(ply_file)
# pprint(vertices)  # Prints the vertex coordinates


In [9]:
import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation as R

def load_ply(ply_path):
    """Loads a PLY file and returns vertices and colors."""
    mesh = o3d.io.read_triangle_mesh(ply_path)
    vertices = np.asarray(mesh.vertices)
    colors = np.asarray(mesh.vertex_colors) if mesh.has_vertex_colors() else None
    return vertices, colors, mesh.triangles

def save_ply(vertices, colors, triangles, output_path):
    """Saves the combined mesh as a PLY file."""
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    if colors is not None:
        mesh.vertex_colors = o3d.utility.Vector3dVector(colors)
    o3d.io.write_triangle_mesh(output_path, mesh)

def find_mean_z_at_xy(vertices, x, y):
    """Finds the mean Z value of all (x, y) points in the given vertices."""
    matching_points = [v[2] for v in vertices if np.isclose(v[0], x) and np.isclose(v[1], y)]
    return np.mean(matching_points) if matching_points else None

def transform_model(vertices, rotation, offset, pivot):
    """Rotates and offsets the model around a pivot point."""
    rx, ry, rz = np.radians(rotation)  # Convert degrees to radians
    rotation_matrix = R.from_euler('xyz', [rx, ry, rz], degrees=False).as_matrix()

    # Translate to pivot, rotate, then translate back
    transformed_vertices = []
    for v in vertices:
        v_shifted = v - pivot  # Move to pivot
        v_rotated = rotation_matrix @ v_shifted  # Apply rotation
        v_transformed = v_rotated + pivot + offset  # Move back and apply offset
        transformed_vertices.append(v_transformed)

    return np.array(transformed_vertices)

def combine_ply_models(ply1, ply2, output_path, rotation, offset, pivot_xy):
    
    pivot_xy=(pivot_xy[0],-pivot_xy[1])
    """Combines two PLY models after transforming the second one."""
    # Load both PLY files
    vertices1, colors1, faces1 = load_ply(ply1)
    vertices2, colors2, faces2 = load_ply(ply2)

    # Set the first model's origin at (0,0,0)
    # pivot_point = np.mean(vertices1, axis=0)

    # Find pivot Z in the second model
    pivot_z = find_mean_z_at_xy(vertices2, *pivot_xy)
    if pivot_z is None:
        raise ValueError(f"Point {pivot_xy} not found in second model!")

    pivot = np.array([*pivot_xy, pivot_z])

    # Transform the second model
    transformed_vertices2 = transform_model(vertices2, rotation, offset, pivot)

    # Merge models
    combined_vertices = np.vstack([vertices1, transformed_vertices2])
    if colors1 is not None and colors2 is not None:
        combined_colors = np.vstack([colors1, colors2])
    else:
        combined_colors = None

    offset_faces2 = np.asarray(faces2) + len(vertices1)  # Adjust face indices
    combined_faces = np.vstack([faces1, offset_faces2])

    # Save combined model
    save_ply(combined_vertices, combined_colors, combined_faces, output_path)




In [10]:
image_name="pyramid.jpg"

image_path=test_input_path+image_name
output_path=test_output_path+image_name

image=Image.open(image_path).convert("RGB")

# image=diffusers.utils.load_image("https://marigoldmonodepth.github.io/images/einstein.jpg")

image=downscale_image(np.array(image),768)

depth=depth_pipe(image)
# depth=depth_pipe(image,num_inference_steps=1,ensemble_size=3,output_uncertainty=True)
# normal=normal_pipe(image)

vis=depth_pipe.image_processor.visualize_depth(depth.prediction)
# nor=normal_pipe.image_processor.visualize_normals(normal.prediction)

vis[0].save(test_output_path+"depth_"+image_name)
# nor[0].save(test_output_path+"normal_"+image_name)

Marigold predictions...:   0%|          | 0/1 [00:00<?, ?it/s]

Diffusion steps...:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
print(type(depth.prediction))
# print(depth.prediction)
depth_array=depth.prediction
depth_array=1-depth_array

depth_array=np.array([[y[0] for y in x] for x in depth_array[0]])

print(depth_array.shape)
print(np.min(depth_array))
print(np.max(depth_array))

<class 'numpy.ndarray'>
(512, 768)
0.0
0.99316406


In [12]:
image_array=np.array(image)


mesh = fill_3d_mesh(depth_array, color_image=image_array, scale=700, threshold=10e-3, euclid_threshold=0.1,apply_colour=False)

image_3d_name="filled_"+image_name[:image_name.rfind('.')]+".ply"
o3d.io.write_triangle_mesh(test_3d_path+image_3d_name, mesh)

print(f"Saved as {image_3d_name}")

100%|██████████| 393171/393171 [00:01<00:00, 199198.02it/s]


Saved as filled_pyramid.ply


In [13]:
# Example usage
combine_ply_models(
    test_3d_path+"filled_pyramid.ply", test_3d_path+"filled_leg_up.ply",test_3d_path+"filled_damn.ply",
    rotation=(0, 360, 0), offset=(0, 700, 0), pivot_xy=(100, 100)
)

print("Saved as filled_damn.ply")

Saved as filled_damn.ply
